In [ ]:
import pandas as pd
import numpy as np

# 1. Sabit Değerleri Tanımlayalım
daily_installs = 20000

# Retention Oranları (D0 her zaman 1.0'dır)
# D0, D1, D3, D7, D14
variant_a_ret = {0: 1.0, 1: 0.53, 3: 0.27, 7: 0.17, 14: 0.06}
variant_b_ret = {0: 1.0, 1: 0.48, 3: 0.25, 7: 0.19, 14: 0.09}

# Gelir Metrikleri
metrics = {
    'A': {'purchase_ratio': 0.0305, 'ecpm': 9.80, 'ad_imp': 2.3, 'price': 1.99},
    'B': {'purchase_ratio': 0.0315, 'ecpm': 10.80, 'ad_imp': 1.6, 'price': 1.99}
}

# 2. Eksik Günleri Tamamlama 
def get_full_retention(ret_dict, max_days=30):
    days = np.arange(max_days + 1)
    known_days = sorted(ret_dict.keys())
    known_rates = [ret_dict[d] for d in known_days]
    # Aradaki günleri lineer tahminle doldurur
    return np.interp(days, known_days, known_rates)

# 3. DAU ve Gelir 
def run_simulation(variant_name, days=15):
    ret_curve = get_full_retention(variant_a_ret if variant_name == 'A' else variant_b_ret)
    m = metrics[variant_name]
    
    total_rev = 0
    dau_at_target_day = 0
    
    # Her gün yeni kullanıcılar gelir ve eskiler retention'a göre kalır
    for day in range(days):
        current_dau = 0
        for i in range(day + 1):
            age = day - i
            current_dau += daily_installs * ret_curve[age]
        
        # Günlük Gelir Hesabı
        daily_iap = current_dau * m['purchase_ratio'] * m['price']
        daily_ad = (current_dau * m['ad_imp'] * m['ecpm']) / 1000
        total_rev += (daily_iap + daily_ad)
        
        if day == days - 1:
            dau_at_target_day = current_dau
            
    return int(dau_at_target_day), round(total_rev, 2)


dau_a, rev_a = run_simulation('A', 15)
dau_b, rev_b = run_simulation('B', 15)

print(f"VARIANT A (15. Gün): DAU = {dau_a}, Toplam Gelir = ${rev_a}")
print(f"VARIANT B (15. Gün): DAU = {dau_b}, Toplam Gelir = ${rev_b}")

In [ ]:
# Sadece Variant B için indirim senaryosu (Purchase Ratio: %3.15 -> %4.15)
def run_sale_simulation(days=10):
    m = metrics['B'].copy()
    m['purchase_ratio'] += 0.01  # %1 artış
    return extra_revenue

In [ ]:
def calculate_sale_impact(variant='B', days=10):
    m = metrics[variant].copy()
    
    # Mevcut 15. gündeki DAU'yu baz alalım 
    current_dau = dau_b 
    
    # 1 İndirim yokken 10 günlük gelir
    normal_daily_iap = current_dau * m['purchase_ratio'] * m['price']
    normal_total_iap = normal_daily_iap * days
    
    # 2 İndirim varken 10 günlük gelir (Purchase Ratio %1 artıyor)
    sale_purchase_ratio = m['purchase_ratio'] + 0.01
    sale_daily_iap = current_dau * sale_purchase_ratio * m['price']
    sale_total_iap = sale_daily_iap * days
    
    # Ekstra gelir
    extra_revenue = sale_total_iap - normal_total_iap
    
    return round(extra_revenue, 2)

extra_cash = calculate_sale_impact('B', 10)
print(f"10 Günlük İndirim Senaryosunda Ekstra Kazanç: ${extra_cash}")

In [ ]:
import math

def new_source_retention(day, variant='B'):
    if day == 0: return 1.0  # İlk gün herkes oyunda
    
    # PDF'teki formüller:
    # Variant A (New): 0.58 * e^(-0.12 * (day-1))
    # Variant B (New): 0.52 * e^(-0.10 * (day-1))
    
    if variant == 'A':
        return 0.58 * math.exp(-0.12 * (day))
    else:
        return 0.52 * math.exp(-0.10 * (day))

# Şimdi (12k eski + 8k yeni) DAU'yu hesaplayalım
def calculate_hybrid_dau(total_days=30, variant='B'):
    old_curve = get_full_retention(variant_a_ret if variant == 'A' else variant_b_ret)
    
    total_dau = 0
    for day in range(total_days):
        # 12.000 eski kaynaktan gelenlerin birikmesi
        old_part = 12000 * old_curve[day]
        # 8.000 yeni kaynaktan gelenlerin birikmesi
        new_part = 8000 * new_source_retention(day, variant)
        total_dau += (old_part + new_part)
        
    return total_dau

In [ ]:


# a, b, c şıkları: 15 ve 30 günlük simülasyon
dau_15_a, rev_15_a = run_simulation('A', 15)
dau_15_b, rev_15_b = run_simulation('B', 15)
dau_30_a, rev_30_a = run_simulation('A', 30)
dau_30_b, rev_30_b = run_simulation('B', 30)

# d şıkkı: İndirim (Sale) 
sale_extra = calculate_sale_impact('B', 10)

# e şıkkı:30. Gün Toplam Gelir
# Bu kısım için yukarıdaki calculate_hybrid_dau fonksiyonunu kullanıyoruz
hybrid_dau_30_b = calculate_hybrid_dau(30, 'B')
# Hibrit geliri basitçe DAU üzerinden tahmin edelim 
arpu_b = rev_30_b / (dau_30_b * 30) # Ortalama günlük kazanç
hybrid_rev_30_b = hybrid_dau_30_b * arpu_b * 30 

print("-" * 30)
print("TASK 1 SONUÇLARI")
print("-" * 30)
print(f"15. GÜN: Variant A DAU: {dau_15_a} | Gelir: ${rev_15_a}")
print(f"15. GÜN: Variant B DAU: {dau_15_b} | Gelir: ${rev_15_b}")
print("-" * 20)
print(f"30. GÜN: Variant A DAU: {dau_30_a} | Gelir: ${rev_30_a}")
print(f"30. GÜN: Variant B DAU: {dau_30_b} | Gelir: ${rev_30_b}")
print("-" * 20)
print(f"D ŞIKKI (10 Günlük İndirim): Ekstra Gelir: ${sale_extra}")
print(f"E ŞIKKI (Yeni Kaynak): 30. Gün Hibrit Toplam Gelir: ${round(hybrid_rev_30_b, 2)}")
print("-" * 30)